In [12]:
import fitz  # PyMuPDF

doc = fitz.open(r"E:\SEMESTER 1\PF~ADIL SHEHZAD\BOOKS\C++ How to Program, 10th edition.pdf")




In [17]:
from IPython.core.interactiveshell import InteractiveShell

# Show all output, not just last expression
InteractiveShell.ast_node_interactivity = "all"

# Increase max output length
from IPython.display import HTML
import sys
sys.stdout.write = lambda x: HTML(x)

In [20]:
for page_num in range(10):#len(doc)):
    page = doc[page_num]
    print(page.get_text())    # Get text with formatting
    text = page.get_text()
    
    # Get text with font info (bold, size, etc.)
    blocks = page.get_text("dict")["blocks"]
    
    # Get images
    images = page.get_images()
    
    print(f"Page {page_num + 1}:")
    print(f"  Text blocks: {len(blocks)}")
    print(f"  Images: {len(images)}")

dumperina

Page 1:
  Text blocks: 2
  Images: 1

Page 2:
  Text blocks: 0
  Images: 1
This page intentionally left blank

Page 3:
  Text blocks: 1
  Images: 0
Paul Deitel
Deitel & Associates, Inc.
Harvey Deitel
Deitel & Associates, Inc.

Page 4:
  Text blocks: 2
  Images: 1
Vice President, Editorial Director: Marcia Horton
Acquisitions Editor: Tracy Johnson
Editorial Assistant: Kristy Alaura
Acquisitions Editor, Global Editions: Sourabh Maheshwari
VP of Marketing: Christy Lesko
Director of Field Marketing: Tim Galligan
Product Marketing Manager: Bram Van Kempen
Field Marketing Manager: Demetrius Hall
Marketing Assistant: Jon Bryant
Director of Product Management: Erin Gregg
Team Lead, Program and Project Management: Scott Disanno
Program Manager: Carole Snyder
Project Manager: Robert Engelhardt
Project Editor, Global Editions: K.K. Neelakantan
Senior Manufacturing Controller, Global Editions: Trudy Kimber
Senior Specialist, Program Planning and Support: Maura Zaldivar-Garcia
Media Prod

In [22]:
import fitz  # PyMuPDF
from PIL import Image
import io



# Extract images from a specific page
page_num = 0  # First page
page = doc[page_num]

# Get all images on the page
images = page.get_images()

print(f"Found {len(images)} images on page {page_num + 1}")

# Extract each image
for img_index, img in enumerate(images):
    # Get image reference
    xref = img[0]
    
    # Extract image bytes
    base_image = doc.extract_image(xref)
    image_bytes = base_image["image"]
    image_ext = base_image["ext"]  # png, jpeg, etc.
    
    # Save image to file
    with open(f"image_page{page_num + 1}_{img_index + 1}.{image_ext}", "wb") as f:
        f.write(image_bytes)
    
    print(f"✓ Saved: image_page{page_num + 1}_{img_index + 1}.{image_ext}")

Found 1 images on page 1


597371

✓ Saved: image_page1_1.jpeg


In [ ]:
import re
from typing import List, Dict
import pdfplumber
import tiktoken

def chunk_pdf_for_rag(
    pdf_path: str,
    max_tokens: int = 500,
    overlap_tokens: int = 75,
    encoding_name: str = "cl100k_base"
) -> List[Dict]:
    """
    Token-accurate, sentence-based chunking for RAG.
    Returns chunks with page range and chapter metadata.
    """

    enc = tiktoken.get_encoding(encoding_name)

    def count_tokens(text: str) -> int:
        return len(enc.encode(text))

    def detect_chapter(text: str):
        match = re.search(r'\b(chapter|chap)\s+(\d+)\b', text, re.IGNORECASE)
        return int(match.group(2)) if match else None

    def split_into_sentences(text: str) -> List[str]:
        return re.split(r'(?<=[.!?])\s+', text.strip())

    chunks = []
    current_chunk = ""
    current_tokens = 0
    current_chapter = None
    chunk_start_page = None

    i = 0
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            raw_text = page.extract_text() or ""
            sentences = split_into_sentences(raw_text)

            for sentence in sentences:
                sentence = sentence.strip()
                if not sentence:
                    continue

                # Detect chapter headings
                chapter = detect_chapter(sentence)
                if chapter is not None and chapter != current_chapter:
                    # Flush chunk on chapter boundary
                    if current_chunk.strip():
                        chunks.append({
                            "text": current_chunk.strip(),
                            "page_start": chunk_start_page,
                            "page_end": page_num,
                            "chapter_index": current_chapter,
                            "chunk_type": "text"
                        })

                    current_chunk = ""
                    current_tokens = 0
                    current_chapter = chapter
                    chunk_start_page = None

                sentence_tokens = count_tokens(sentence)
                # Initialize chunk start page
                if current_chunk == "":
                    chunk_start_page = page_num

                # FIX: Check if adding this sentence would exceed limit
                if current_tokens + sentence_tokens > max_tokens:
                    # Emit current chunk (only if not empty)
                    if current_chunk.strip():
                        chunks.append({
                            "text": current_chunk.strip(),
                            "page_start": chunk_start_page,
                            "page_end": page_num,
                            "chapter_index": current_chapter,
                            "chunk_type": "text"
                        })

                    # Create overlap from previous chunk
                    if current_chunk.strip():
                        overlap_text = enc.decode(
                            enc.encode(current_chunk)[-overlap_tokens:]
                        )
                    else:
                        overlap_text = ""

                    # Start new chunk with overlap + current sentence
                    current_chunk = (overlap_text + " " + sentence).strip() + " "
                    current_tokens = count_tokens(current_chunk)
                    chunk_start_page = page_num
                    
                    # FIX: If even the new chunk with overlap exceeds limit, 
                    # start fresh without overlap
                    if current_tokens > max_tokens:
                        current_chunk = sentence + " "
                        current_tokens = sentence_tokens
                else:
                    # Add sentence to current chunk
                    current_chunk += sentence + " "
                    current_tokens += sentence_tokens
                
    if current_chunk.strip():
        chunks.append({
            "text": current_chunk.strip(),
            "page_start": chunk_start_page,
            "page_end": page_num,
            "chapter_index": current_chapter,
            "chunk_type": "text"
        })

    return chunks

In [ ]:
path = "E:\SEMESTER 1\PF~ADIL SHEHZAD\BOOKS\C++ How to Program, 10th edition.pdf"

chunks= chunk_pdf_for_rag(path)

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\hazzy\AppData\Local\Temp\ipykernel_21828\999815350.py:1: SyntaxWarning: invalid escape sequence '\S'
  path = "E:\SEMESTER 1\PF~ADIL SHEHZAD\BOOKS\C++ How to Program, 10th edition.pdf"


Processing 1076 pages with 4 processes:
  Process 0: Pages 1-269
  Process 1: Pages 270-538
  Process 2: Pages 539-807
  Process 3: Pages 808-1076


In [78]:
for chk in chunks[ 50: 55]: 
    print(len(chk['text']))